<h2 style="font-size:24pt"> Proyecto DESI</h2>

<h2 style="font-size:24pt"> Julio 10, 2025</h2>

<p style="font-size:16pt">
For each rosette, 100 random data samples are saved.

In [1]:
from astropy.table import Table, vstack
import pandas as pd
import numpy as np
from astropy.cosmology import Planck18
from astropy.coordinates import SkyCoord
from astropy import units as u
from astropy.io import ascii

## Real data

In [2]:
data_n = Table.read("https://data.desi.lbl.gov/public/edr/vac/edr/lss/v2.0/LSScats/clustering/LRG_N_clustering.dat.fits")
data_s = Table.read("https://data.desi.lbl.gov/public/edr/vac/edr/lss/v2.0/LSScats/clustering/LRG_S_clustering.dat.fits")

names_n = [name for name in data_n.colnames if len(data_n[name].shape) <= 1]
names_s = [name for name in data_n.colnames if len(data_n[name].shape) <= 1]

data_n_filt = data_n[names_n].to_pandas()
data_s_filt = data_s[names_s].to_pandas()

In [3]:
sizes_n = {}
for i in set(data_n_filt['ROSETTE_NUMBER']):
    df_rosette = data_n_filt[data_n_filt['ROSETTE_NUMBER']==i]
    sizes_n[i] = len(df_rosette)

sizes_s = {}
for i in set(data_s_filt['ROSETTE_NUMBER']):
    df_rosette = data_s_filt[data_s_filt['ROSETTE_NUMBER']==i]
    sizes_s[i] = len(df_rosette)

In [16]:
set(data_s_filt['ROSETTE_NUMBER'])

{0, 1, 2, 4, 5, 8, 9, 10, 16, 17}

## Random

In [5]:
def cartesianas(data):
    data['distance'] = Planck18.comoving_distance(data['Z'])
    coords = SkyCoord(ra=data['RA'], dec=data['DEC'], distance=data['distance'], frame='icrs')
    data['x'] = coords.cartesian.x.value
    data['y'] = coords.cartesian.y.value
    data['z'] = coords.cartesian.z.value

    data['x'] = data['x'] * u.Mpc
    data['y'] = data['y'] * u.Mpc
    data['z'] = data['z'] * u.Mpc

In [12]:
def url(rand_generator, orientation):
    if orientation == 'N':
        urls = [
            f"https://data.desi.lbl.gov/public/edr/vac/edr/lss/v2.0/LSScats/clustering/LRG_N_{i}_clustering.ran.fits"
            for i in range(18)
        ]
    elif orientation == 'S':
        urls = [
            f"https://data.desi.lbl.gov/public/edr/vac/edr/lss/v2.0/LSScats/clustering/LRG_S_{i}_clustering.ran.fits"
            for i in range(18)
        ]
    url = rand_generator.choice(urls)
    rand = Table.read(url)
    return rand

def save_file(data, sizes, orientation, n_random=100, seed=42):
    
    rosettes = np.unique(data['ROSETTE_NUMBER'])
    rng = np.random.default_rng(seed) 
    rand = url(rng,orientation)
    
    for i in rosettes:
        rand_table_rosette = rand[rand['ROSETTE_NUMBER'] == i]
    
        for j in range(n_random):
            rng_j = np.random.default_rng(seed + i * 100 + j)
            idx_random = rng_j.choice(len(rand_table_rosette), size=sizes[i], replace=False)
            rand_subset = rand_table_rosette[idx_random]
    
            cartesianas(rand_subset)
    
            filename_rand = f"rand_rosette/LRG_rosette_{i}_random_{j}.ecsv"
            ascii.write(rand_subset, filename_rand, format='ecsv', overwrite=True)

In [13]:
%%time
save_file(data_n_filt,sizes_n,'N')

CPU times: total: 7min 38s
Wall time: 8min 3s


In [14]:
%%time
save_file(data_s_filt,sizes_s,'S')

CPU times: total: 6min 35s
Wall time: 6min 50s
